In [1]:
"""
Positive and negative acceptor and donor are available by some previous processes. 
Generate kmer version of those data.
"""
from data_preparation import generate_kmer_csv
from data_dir import ss_pos_acc_hs_csv, ss_pos_don_hs_csv, ss_neg_acc_hs_csv, ss_neg_don_hs_csv, ss_pos_acc_hs_kmer_csv, ss_pos_don_hs_kmer_csv, ss_neg_acc_hs_kmer_csv, ss_neg_don_hs_kmer_csv

file_pairs = [
    (ss_pos_acc_hs_csv, ss_pos_acc_hs_kmer_csv),
    (ss_pos_don_hs_csv, ss_pos_don_hs_kmer_csv),
    (ss_neg_acc_hs_csv, ss_neg_acc_hs_kmer_csv),
    (ss_neg_don_hs_csv, ss_neg_don_hs_kmer_csv)
]

for src, target_dir in file_pairs:
    generate_kmer_csv(src, target_dir)

In [3]:
"""
Generate kmer version of splice site data.
"""
from data_dir import ss_dir
from data_preparation import generate_kmer_csv
src_train = "{}/train_no_kmer.csv".format(ss_dir)
src_validation = "{}/validation_no_kmer.csv".format(ss_dir)
src_test = "{}/test_no_kmer.csv".format(ss_dir)

target_train = "{}/train.csv".format(ss_dir)
target_validation = "{}/validation.csv".format(ss_dir)
target_test = "{}/test.csv".format(ss_dir)

_pairs = [(src_validation, target_validation), (src_test, target_test)]
#for src, target in _pairs:
#    print("Generate kmer csv for {} => {}: {}".format(src, target, generate_kmer_csv(src, target)))

In [4]:
_pairs = [(src_validation, target_validation), (src_test, target_test)]
for src, target in _pairs:
    print("Generate kmer csv for {} => {}: {}".format(src, target, generate_kmer_csv(src, target)))

Generate kmer csv for ./data/splice-sites/splice-deep/validation_no_kmer.csv => ./data/splice-sites/splice-deep/validation.csv: True
Generate kmer csv for ./data/splice-sites/splice-deep/test_no_kmer.csv => ./data/splice-sites/splice-deep/test.csv: True


In [1]:
"""
Expand Splice-sites and store the result into dataset folder.
"""
from data_dir import ss_dir, dataset_full_ss_dir
from data_preparation import expand_by_sliding_window_no_pandas
_files = ["train.csv", 'validation.csv', 'test.csv']
for fname in _files:
    src_csv = "{}/{}".format(ss_dir, fname)
    target_csv = "{}/{}".format(dataset_full_ss_dir, fname)
    print("Expanding {} => {}: {}".format(src_csv, target_csv, expand_by_sliding_window_no_pandas(src_csv, target_csv, length=510)))

Error [Errno 28] No space left on device
Error Traceback (most recent call last):
  File "w:\Research\_sequence-processing\data_preparation.py", line 1034, in expand_by_sliding_window_no_pandas
    if _count < _len:
OSError: [Errno 28] No space left on device

Expanding ./data/splice-sites/splice-deep/train.csv => ./dataset/full/splice-sites/train.csv: False
Error [Errno 28] No space left on device
Error Traceback (most recent call last):
  File "w:\Research\_sequence-processing\data_preparation.py", line 1034, in expand_by_sliding_window_no_pandas
    if _count < _len:
OSError: [Errno 28] No space left on device

Expanding ./data/splice-sites/splice-deep/validation.csv => ./dataset/full/splice-sites/validation.csv: False
Error [Errno 28] No space left on device
Error Traceback (most recent call last):
  File "w:\Research\_sequence-processing\data_preparation.py", line 1034, in expand_by_sliding_window_no_pandas
    if _count < _len:
OSError: [Errno 28] No space left on device

Expandi

In [2]:
"""
Checking the size of dataset. It appears that generating expanded sequence resulted in too-large file.
"""
from data_dir import ss_neg_don_hs_csv, ss_neg_acc_hs_csv, ss_pos_acc_hs_csv, ss_pos_don_hs_csv
import pandas as pd

ss_neg_acc_df = pd.read_csv(ss_neg_acc_hs_csv)
ss_neg_don_df = pd.read_csv(ss_neg_don_hs_csv)
ss_pos_acc_df = pd.read_csv(ss_pos_acc_hs_csv)
ss_pos_don_df = pd.read_csv(ss_pos_don_hs_csv)

print("ss pos acc {}".format(len(ss_pos_acc_df)))
print("ss pos don {}".format(len(ss_pos_don_df)))
print("ss neg acc {}".format(len(ss_neg_acc_df)))
print("ss neg don {}".format(len(ss_neg_don_df)))

ss pos acc 248150
ss pos don 250400
ss neg acc 248150
ss neg don 250400


In [1]:
"""
Both positive and negative splice site data is too large. This code is for sampling.
Sampling is done because local storage limitation.
"""
from data_preparation import generate_sample
from data_dir import (
    ss_neg_acc_hs_non_kmer_csv, ss_neg_don_hs_non_kmer_csv, ss_pos_acc_hs_non_kmer_csv, ss_pos_don_hs_non_kmer_csv,
    ss_neg_acc_hs_csv, ss_neg_don_hs_csv, ss_pos_acc_hs_csv, ss_pos_don_hs_csv
)

_pairs = [(ss_neg_acc_hs_csv, ss_neg_acc_hs_non_kmer_csv), (ss_neg_don_hs_csv, ss_neg_don_hs_non_kmer_csv), (ss_pos_acc_hs_csv, ss_pos_acc_hs_non_kmer_csv), (ss_pos_don_hs_csv, ss_pos_don_hs_non_kmer_csv)]
for src, target in _pairs:
    print("Generate sample for {} => {}: {}".format(src, target, generate_sample(src, target, n_sample=500)))

Generate sample for ./data/splice-sites/splice-deep/neg_ss_acc_hs.csv => ./data/splice-sites/splice-deep/neg_ss_acc_hs_non_kmer.csv: ./data/splice-sites/splice-deep/neg_ss_acc_hs_non_kmer.csv
Generate sample for ./data/splice-sites/splice-deep/neg_ss_don_hs.csv => ./data/splice-sites/splice-deep/neg_ss_don_hs_non_kmer.csv: ./data/splice-sites/splice-deep/neg_ss_don_hs_non_kmer.csv
Generate sample for ./data/splice-sites/splice-deep/pos_ss_acc_hs.csv => ./data/splice-sites/splice-deep/pos_ss_acc_hs_non_kmer.csv: ./data/splice-sites/splice-deep/pos_ss_acc_hs_non_kmer.csv
Generate sample for ./data/splice-sites/splice-deep/pos_ss_don_hs.csv => ./data/splice-sites/splice-deep/pos_ss_don_hs_non_kmer.csv: ./data/splice-sites/splice-deep/pos_ss_don_hs_non_kmer.csv


In [2]:
"""
Generate kmer version for splice sites data for positive and negative acceptor and donor.
"""
from data_preparation import generate_kmer_csv
from data_dir import (
    ss_neg_acc_hs_non_kmer_csv, ss_neg_don_hs_non_kmer_csv, ss_pos_acc_hs_non_kmer_csv, ss_pos_don_hs_non_kmer_csv,
    ss_neg_acc_hs_kmer_csv, ss_neg_don_hs_kmer_csv, ss_pos_acc_hs_kmer_csv, ss_pos_don_hs_kmer_csv,
)

_pairs = [(ss_neg_acc_hs_non_kmer_csv, ss_neg_acc_hs_kmer_csv), (ss_neg_don_hs_non_kmer_csv, ss_neg_don_hs_kmer_csv), (ss_pos_acc_hs_non_kmer_csv, ss_pos_acc_hs_kmer_csv), (ss_pos_don_hs_non_kmer_csv, ss_pos_don_hs_kmer_csv)]
for src, target in _pairs:
    print("Generate kmer csv for {} => {}: {}".format(src, target, generate_kmer_csv(src, target)))

Generate kmer csv for ./data/splice-sites/splice-deep/neg_ss_acc_hs_non_kmer.csv => ./data/splice-sites/splice-deep/neg_ss_acc_hs_kmer.csv: True
Generate kmer csv for ./data/splice-sites/splice-deep/neg_ss_don_hs_non_kmer.csv => ./data/splice-sites/splice-deep/neg_ss_don_hs_kmer.csv: True
Generate kmer csv for ./data/splice-sites/splice-deep/pos_ss_acc_hs_non_kmer.csv => ./data/splice-sites/splice-deep/pos_ss_acc_hs_kmer.csv: True
Generate kmer csv for ./data/splice-sites/splice-deep/pos_ss_don_hs_non_kmer.csv => ./data/splice-sites/splice-deep/pos_ss_don_hs_kmer.csv: True


In [4]:
"""
Merge all positive acceptor and donor, and negative acceptor and donor into single file for training.
"""
from data_dir import ss_neg_acc_hs_kmer_csv, ss_neg_don_hs_kmer_csv, ss_pos_acc_hs_kmer_csv, ss_pos_don_hs_kmer_csv
from data_dir import ss_train_csv
from data_preparation import merge_csv

files = [ss_neg_acc_hs_kmer_csv, ss_neg_don_hs_kmer_csv, ss_pos_acc_hs_kmer_csv, ss_pos_don_hs_kmer_csv]
print("Merging {} into {}: {}".format(files, ss_train_csv, merge_csv(files, ss_train_csv)))

Merging ['./data/splice-sites/splice-deep/neg_ss_acc_hs_kmer.csv', './data/splice-sites/splice-deep/neg_ss_don_hs_kmer.csv', './data/splice-sites/splice-deep/pos_ss_acc_hs_kmer.csv', './data/splice-sites/splice-deep/pos_ss_don_hs_kmer.csv'] into ./data/splice-sites/splice-deep/train.csv: True


In [ ]:
"""
Expand training data (train.csv) and store the expanded file in dataset full ss folder.
"""